In [1895]:
import pandas as pd
import numpy as np

data = pd.read_csv('./동별통합데이터.csv')
df_raw = pd.DataFrame(data)

# 데이터 확인

df_raw.head(1)
print(df_raw.columns)
print(df_raw.shape)

Index(['Unnamed: 0', 'Gu', 'Dong', 'Hospital_Counts', 'Sales_Counts',
       'Consignment_Counts', 'Transit_Counts', 'Production_Counts',
       'Import_Counts', 'Population', 'Pet_Owners', 'Total_Registrations'],
      dtype='object')
(145, 12)


In [1896]:
# 표준화 패키지 라이브러리 
from sklearn.preprocessing import StandardScaler  

# 독립변인들의 value값만 추출       
x = data[['Hospital_Counts', 'Transit_Counts', 'Population']].values

# 종속변인 추출
y = data['Total_Registrations'].values # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = ['Hospital_Counts', 'Transit_Counts', 'Population']

pd.DataFrame(x, columns=features).head()


,Hospital_Counts,Transit_Counts,Population
0,0.085263,-0.719800,0.074613
1,-0.189473,-0.719800,-0.409129
2,-0.464210,-0.083392,-0.230630
3,2.557892,-0.401596,1.170657
4,-0.464210,-0.401596,-0.441903


In [1898]:
from sklearn.decomposition import PCA


# 주성분을 1개로 선택
pca1 = PCA(n_components=1)
printcipalComponents1 = pca1.fit_transform(x)

# 주성분으로 이루어진 데이터 프레임 구성
principalDf1 = pd.DataFrame(data=printcipalComponents1, columns = ['principal component1'])



# 주성분을 2개로 선택
pca2 = PCA(n_components=2)
printcipalComponents2 = pca2.fit_transform(x)

# 주성분으로 이루어진 데이터 프레임 구성
principalDf2 = pd.DataFrame(data=printcipalComponents2, columns = ['principal component1', 'principal component2'])



# 주성분을 3개로 선택
pca3 = PCA(n_components=3) # 주성분을 몇개로 할지 결정
printcipalComponents3 = pca3.fit_transform(x)

# 주성분으로 이루어진 데이터 프레임 구성
principalDf3 = pd.DataFrame(data=printcipalComponents3, columns = ['principal component1', 'principal component2',
                                                                 'principal component3',])

In [1899]:
print('1개의 주성분 분산 설명량',pca1.explained_variance_ratio_)
print('2개의 주성분 분산 설명량',pca2.explained_variance_ratio_)
print('3개의 주성분 분산 설명량',pca3.explained_variance_ratio_)

1개의 주성분 분산 설명량 [0.81991857]
2개의 주성분 분산 설명량 [0.81991857 0.12338166]
3개의 주성분 분산 설명량 [0.81991857 0.12338166 0.05669977]


In [1900]:
print('주성분 1개, 누적 설명 분산 비율',sum(pca1.explained_variance_ratio_))
print('주성분 2개, 누적 설명 분산 비율',sum(pca2.explained_variance_ratio_))
print('주성분 3개, 누적 설명 분산 비율',sum(pca3.explained_variance_ratio_))


주성분 1개, 누적 설명 분산 비율 0.8199185691628296
주성분 2개, 누적 설명 분산 비율 0.9433002274032503
주성분 3개, 누적 설명 분산 비율 1.0


3개의 독립변수 중, 결론 주성분 3개 선택

In [1902]:
from sklearn.preprocessing import StandardScaler  

# 독립변인들의 value값만 추출       
x = data[['Hospital_Counts', 'Transit_Counts','Population']].values

# 종속변인 추출
y = data['Total_Registrations'].values # 종속변인 추출

x = StandardScaler().fit_transform(x) # x객체에 x를 표준화한 데이터를 저장

features = ['Hospital_Counts', 'Transit_Counts','Population']


pd.DataFrame(x, columns=features).head()



# 주성분을 3개로 선택
pca = PCA(n_components=3) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(x)

# 주성분으로 이루어진 데이터 프레임 구성
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1','principal component2','principal component3'])


# train-test data set(0.8:0.2)
import os
from sklearn.model_selection import train_test_split

x = principalDf
y = y

model2_train_x_v4, model2_test_x_v4, model2_train_y_v4, model2_test_y_v4 = train_test_split(x,y, test_size =0.2, random_state=423)
print(model2_train_x_v4.shape, model2_test_x_v4.shape, model2_train_y_v4.shape, model2_test_y_v4.shape) 


(116, 3) (29, 3) (116,) (29,)


In [1903]:
import numpy as np
import pandas as pd

# 평가 지표 계산 함수
def score(test_y, predict):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    mse = mean_squared_error(test_y, predict)
    mae = mean_absolute_error(test_y, predict)
    r_squared = r2_score(test_y, predict)
    rmse = mean_squared_error(test_y, predict, squared=False)

    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    print("R-squared:", r_squared)
    print("RMSE:", rmse)


In [1904]:
import statsmodels.api as sm

# 예시 데이터 생성
x = model2_train_x_v4
y = model2_train_y_v4

# 로버스트 회귀 모델 적합
X_with_const = sm.add_constant(x)  # 상수항 추가
model_robust = sm.RLM(y, X_with_const, M=sm.robust.norms.HuberT())
result_robust = model_robust.fit()

# 결과 출력
predict_robust = result_robust.predict(sm.add_constant(model2_test_x_v4))

# 미세조정

predict_robust = np.clip(predict_robust, 0, 9894.280840991367)

# 평가지표 계산
score(model2_test_y_v4, predict_robust)


# 종속변수 3개 주 성분 3개 랜덤 316//R-squared: R-squared: 0.9424816381216994

Mean Squared Error (MSE): 562749.660052271
Mean Absolute Error (MAE): 508.1852847115225
R-squared: 0.9156369975622975
RMSE: 750.1664215707546


In [1905]:
from sklearn.linear_model import Ridge

alpha = 0.9  # 규제 강도 (하이퍼파라미터)
model_ridge = Ridge(alpha=alpha)
x = model2_train_x_v4
y = model2_train_y_v4
# 모델 학습
model_ridge.fit(x,y)

# 테스트 데이터로 예측
predict_ridge = model_ridge.predict(model2_test_x_v4)

# 평가지표 계산
score(model2_test_y_v4, predict_ridge)


# random_state=423


Mean Squared Error (MSE): 311842.1324716351
Mean Absolute Error (MAE): 435.5419943120359
R-squared: 0.9532510804547811
RMSE: 558.4282697640182


In [1906]:
from sklearn.linear_model import ElasticNet

# 엘라스틱넷 회귀 모델 생성

alpha = 7.0 # L1 및 L2 규제 강도 (하이퍼파라미터)
l1_ratio = 0.1  # L1 규제의 비율 (하이퍼파라미터)
model_el = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)

x = model2_train_x_v4
y = model2_train_y_v4

# 모델 학습
model_el.fit(x,y)

# 테스트 데이터로 예측
predict_el = model_el.predict(model2_test_x_v4)

# 미세조정
predict_el = np.clip(predict_el, 0, 9894.280840991367)

# 평가지표 계산
score(model2_test_y_v4, predict_el)

# 1 (0.2)0.3896202025220652

Mean Squared Error (MSE): 3776858.2185055893
Mean Absolute Error (MAE): 1409.1759041535247
R-squared: 0.4338031247054899
RMSE: 1943.4140625470397


In [1907]:
from sklearn.linear_model import LinearRegression

model_linear= LinearRegression()

x = model2_train_x_v4
y = model2_train_y_v4

# 모델 적합
result_linear = model_linear.fit(x, y)

# 결과 출력
predict_linear = result_linear.predict(model2_test_x_v4)

# 미세조정
predict_linear = np.clip(predict_linear, 0, 9336.715817321081)

# 평가지표 계산
score(model2_test_y_v4, predict_linear)

# 1 (0.2)0.7343438238428617
#2 (0.2)R-squared: 0.8923678719936224(42)

Mean Squared Error (MSE): 646003.4168905976
Mean Absolute Error (MAE): 516.4505538725757
R-squared: 0.9031562492124056
RMSE: 803.7433775096362


In [1908]:
import statsmodels.api as sm

# 예시 데이터 생성
x = model2_train_x_v4
y = model2_train_y_v4

# 로버스트 회귀 모델 적합
X_with_const = sm.add_constant(x)  # 상수항 추가
model_robust = sm.RLM(y, X_with_const, M=sm.robust.norms.HuberT())
result_robust = model_robust.fit()

# 결과 출력
predict_robust = result_robust.predict(sm.add_constant(model2_test_x_v4))

# 미세조정

#predict_robust = np.clip(predict_robust, 0, 9605.5158)
predict_robust = np.clip(predict_robust, 0, 9894.280840991367)

# 평가지표 계산
score(model2_test_y_v4, predict_robust)

# 1 (0.2)0.7487317587432627
# 2 (0.2)0.890167744003798
# 독립변수 변경 2개 주 속성 0.2 R-squared: 0.896001959824208
#R-squared: 0.8965951385161472(9605.52)

Mean Squared Error (MSE): 562749.660052271
Mean Absolute Error (MAE): 508.1852847115225
R-squared: 0.9156369975622975
RMSE: 750.1664215707546


In [1909]:
from sklearn.ensemble import RandomForestRegressor

# 데이터 준비 (X: 독립 변수, y: 종속 변수)
x = model2_train_x_v4
y = model2_train_y_v4

# 랜덤 포레스트 회귀 모델 생성
model_rf = RandomForestRegressor(n_estimators=1000, max_depth=5, random_state=800)

# 모델 학습
model_rf.fit(x, y)

# 예측
predict_rf = model_rf.predict(model2_test_x_v4)

# 평가지표 계산
score(model2_test_y_v4, predict_rf)

# 1 (0.2)0.6636164446164734
# 2 (0.2)0.7077178515763327
# 독립변수 4개 R-squared: 0.7971833601738879

Mean Squared Error (MSE): 592924.4736925383
Mean Absolute Error (MAE): 513.8835899351859
R-squared: 0.911113426856889
RMSE: 770.0158918441479
